# How People Decide what they want to do
Directed graph approach

People (`pops`) have desires for things like wealth, science and industry. 

## Action
An anonymous function will choose a pop and run the following actions:
* decide which actions, if any, that pop is capable of
* if there is at least one desire with met requirements
    * the pop will take the action with the greatest desire
* if there isn't at least one action
    * desire for one of those items will increase



**Note** this notebook actualy builds the desires into the graph, overwriting existing ontology. Nodes that do this are commented out.



In [7]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [8]:
local_user = "BillmanLocal2"
res = db.run_query(f"g.V().hasLabel('pop').has('username','{local_user}').valueMap()")
pops = [db.clean_node(n) for n in res]
pops[0]

{'conformity': 0.199,
 'literacy': 0.722,
 'aggression': 0.508,
 'constitution': 0.481,
 'objid': '4253777177342',
 'faction_no': 0,
 'name': 'Roysa Ta',
 'isInFaction': '3323278109620',
 'industry': 0.4945,
 'wealth': 0.6082,
 'faction_loyalty': 0.5225,
 'username': 'BillmanLocal2',
 'objtype': 'pop',
 'id': '4253777177342'}

Each population wants to do everything to a degree, the amount of desire to do that thing is expressed by the edge weight. 
* Attack a population
* Focus on improving literacy
* Focus on improving industry

# Desires as Objects <a class="anchor" id="desire"></a>

## Desire with targets.
Both factions and pops can have desire. Action is guided by desire based on the `max(desire.weight)`.
`desire` is an edge, the type of that desire is a property of that edge, and the edge weight is the amount of desire. The target (`node2`) is the recipient. 

Examples:
* faction wants trade with faction
* pop wants war with another pop
* pop wants faction to go to war with faction

## Desire without targets.

Desires without targets must link to an objective. That objective can be it's own node.

### This next cell will upload new desires from `desires.yaml`, which can be edited at any time. 

desires.yaml can be used to configure how people act as a default. This can be helpful in adjusting the default settings in your universe. 


In [9]:
# # Drop the items, if they exist. 
# db.run_query("g.V().hasLabel('objective').has('username','notebook').drop()")
# objectives_yaml = yaml.safe_load(open("desires.yaml"))['objectives']
# data = {"nodes":objectives_yaml,'edges':[]}
# # Then Create the nodes and add them to the DB
# db.upload_data(db.c,data,verbose=False)
# After creating the nodes, pulling them into the notebook for reference
# I won't re-create them here, unless I've updated the file. 
res = db.run_query("g.V().hasLabel('objective').valueMap()")
objectives = [db.clean_node(n) for n in res]
pd.DataFrame(objectives)


,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


The same for actions:

In [10]:
# Drop the items, if they exist. 
# db.run_query("g.V().hasLabel('action').has('username','notebook').drop()")
# actions_yaml = yaml.safe_load(open("actions_pop.yaml"))['actions']
# data = {"nodes":actions_yaml,'edges':[]}

# # Then Create the nodes and add them to the DB
# db.upload_data(db.c,data,verbose=True)

# After creating the nodes, pulling them into the notebook for reference
# I won't re-create them here, unless I've updated the file. 
res = db.run_query("g.V().hasLabel('action').valueMap()")
action = [db.clean_node(n) for n in res]
pd.DataFrame(action)


,type,required_desire,leadingdesire,comment,username,objtype,objid,id
0,patriot_education,0.5,wealth,expand public education programs with a collec...,notebook,action,6215825744107,6215825744107
1,safetynets,0.7,wealth,increase the wealth and literacy of other popu...,notebook,action,7818338629871,7818338629871
2,charity,0.8,wealth,increase the wealth and literacy of population...,notebook,action,7100369067332,7100369067332
3,attack,0.8,aggression,"pointless loss of lives and wealth, possibly r...",notebook,action,6881021997118,6881021997118
4,patriot_propoganda,0.2,industry,"build national pride, increasin faction loyalt...",notebook,action,8960412060009,8960412060009
5,betterliving,0.3,literacy,researching advances in medication and domesti...,notebook,action,9395913208376,9395913208376
6,individual_education,0.2,wealth,"expand privatized education, increase literacy...",notebook,action,1129902930373,1129902930373
7,buildinfra,0.5,wealth,increase industry by investing in local infras...,notebook,action,2507865371045,2507865371045


# population wants to improve industry

populations want to improve industry when:
* they are not wealty
* they are at war




In [11]:
# Marginal return on base attribute
n = 2
ind_df = pd.DataFrame(np.sort([float(p['wealth']) for p in pops]),columns=['wealth'])
ind_df['base'] = range(len(ind_df))
ind_df['desires_industry'] = ind_df['wealth'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
ind_df['desire_base'] = ind_df['base'].apply(lambda x: ((x+1)**(1-n) - 1)/(1-n))
alt.Chart(ind_df).mark_line().encode(x='base',y='desire_base').properties(title="Desire relative to the base attribute")

alt.Chart(...)

In [12]:
alt.Chart(ind_df).mark_line().encode(x='wealth:N',y='desires_industry').properties(title="Desires wealth industry relative to industry")

alt.Chart(...)

## feeding that desire to the populations

Per above, the initial desire to do a thing is dependant on the `leading attribute`. This makes it easy to adjust the population desires in the `desires.yaml`.

In [13]:
def get_desire(x):
       return np.round(((float(x)+1)**(1-n) - 1)/(1-n),3)

popno = 0
objectiveno = 0

print(pops[popno])
print(objectives[objectiveno])

{'conformity': 0.199, 'literacy': 0.722, 'aggression': 0.508, 'constitution': 0.481, 'objid': '4253777177342', 'faction_no': 0, 'name': 'Roysa Ta', 'isInFaction': '3323278109620', 'industry': 0.4945, 'wealth': 0.6082, 'faction_loyalty': 0.5225, 'username': 'BillmanLocal2', 'objtype': 'pop', 'id': '4253777177342'}
{'type': 'industry', 'weight': '0.5', 'leadingAttribute': 'wealth', 'comment': 'build factories, skyscrapers, infrastructure', 'username': 'notebook', 'objtype': 'objective', 'objid': '2008795542230', 'id': '2008795542230'}


In [14]:
get_desire(pops[popno][objectives[objectiveno]['leadingAttribute']])

0.378

Create a desire edge for each desire, for each population.

In [15]:
edges = []
for p in pops:
    for o in objectives:
        edge = {'label':'desires',
                'node1':p['objid'],
                'node2':o['objid'],
                'desire':o['type'],
                'weight':get_desire(p[o['leadingAttribute']])}
        edges.append(edge)

In [16]:
o

{'type': 'science',
 'weight': '0.5',
 'leadingAttribute': 'literacy',
 'comment': 'building schools, education systems, information infrastructure',
 'username': 'notebook',
 'objtype': 'objective',
 'objid': '7056154012146',
 'id': '7056154012146'}

# Weighing based on the strongest desire

Here is how you take one pop, and calculate what it desires most. 

In [17]:
query = "g.V().hasLabel('objective').valueMap()"
db.qtodf(query)

,type,weight,leadingAttribute,comment,username,objtype,objid,id
0,industry,0.5,wealth,"build factories, skyscrapers, infrastructure",notebook,objective,2008795542230,2008795542230
1,expansion,0.5,industry,"to increase population, eventually becoming a ...",notebook,objective,3832546566671,3832546566671
2,war,0.4,aggression,the general amount that the society wants to f...,notebook,objective,9111777612067,9111777612067
3,wealth,0.5,literacy,"increasing luxury, entertainiment and amenities",notebook,objective,9171875985828,9171875985828
4,science,0.5,literacy,"building schools, education systems, informati...",notebook,objective,7056154012146,7056154012146


In [ ]:
db.qtodf(f"g.V().hasLabel('pop').has('username','{local_user}').limit(2).valueMap()")

Arbitrarily grabbing a pop id: `9033676237817`

In [18]:
popid1 = "4253777177342"
popid2 = "8383433262758"

I'll likely revisit this query, but it moves the project along for now. This returns the desire for each objective. This can be expanded as it grows. 

In [31]:
def flatten(list_of_lists):
    if len(list_of_lists) == 0:
        return list_of_lists
    if isinstance(list_of_lists[0], list):
        return flatten(list_of_lists[0]) + flatten(list_of_lists[1:])
    return list_of_lists[:1] + flatten(list_of_lists[1:])

d_map = db.run_query(f"g.V().has('objid','{popid2}').outE('desires').inV().dedup().path().by(values('name','objid').fold()).by('weight').by(values('type','objid','comment','leadingAttribute').fold())")
desires_list = [flatten(d['objects']) for d in d_map]
df_desires = pd.DataFrame(desires_list, columns=['name','objid','weight','type','objid','comment','leadingAttribute'])
df_desires

,name,objid,weight,type,objid,comment,leadingAttribute
0,Renna Ditan,8383433262758,0.389,industry,2008795542230,"build factories, skyscrapers, infrastructure",wealth
1,Renna Ditan,8383433262758,0.415,expansion,3832546566671,"to increase population, eventually becoming a ...",industry
2,Renna Ditan,8383433262758,0.360,wealth,9171875985828,"increasing luxury, entertainiment and amenities",literacy
3,Renna Ditan,8383433262758,0.362,war,9111777612067,the general amount that the society wants to f...,aggression
4,Renna Ditan,8383433262758,0.360,science,7056154012146,"building schools, education systems, informati...",literacy


In [ ]:
columns=['name','objid','weight','type','objid','comment','leadingAttribute']
[{columns[j[0]]:j[1] for j in enumerate(i) if columns[j[0]]!='objid'} for i in regular_list]

# User actions

Specifically actions that are taken by the user (not things pops do by themselves)
Actions have a relationship with desire.
* the `leadingdesire` shows which desire warrants that action
* the `required_desire` shows how much they need to want it for it to be available. 

In [27]:
res = db.run_query("g.V().hasLabel('action').valueMap()")
action = [db.clean_node(n) for n in res]
df_action = pd.DataFrame(action)
df_action

,type,required_desire,leadingdesire,comment,username,objtype,objid,id
0,patriot_education,0.5,wealth,expand public education programs with a collec...,notebook,action,6215825744107,6215825744107
1,safetynets,0.7,wealth,increase the wealth and literacy of other popu...,notebook,action,7818338629871,7818338629871
2,charity,0.8,wealth,increase the wealth and literacy of population...,notebook,action,7100369067332,7100369067332
3,attack,0.8,aggression,"pointless loss of lives and wealth, possibly r...",notebook,action,6881021997118,6881021997118
4,patriot_propoganda,0.2,industry,"build national pride, increasin faction loyalt...",notebook,action,8960412060009,8960412060009
5,betterliving,0.3,literacy,researching advances in medication and domesti...,notebook,action,9395913208376,9395913208376
6,individual_education,0.2,wealth,"expand privatized education, increase literacy...",notebook,action,1129902930373,1129902930373
7,buildinfra,0.5,wealth,increase industry by investing in local infras...,notebook,action,2507865371045,2507865371045


But when we filter the total available actions by the ones that are available

In [59]:
action_filter = (df_action[['type','required_desire']]
    .apply(
        lambda x: df_desires[df_desires['type']=='industry']['weight']
        .values[0]>x.required_desire,
        axis=1
        ))
df_action[action_filter]

,type,required_desire,leadingdesire,comment,username,objtype,objid,id
4,patriot_propoganda,0.2,industry,"build national pride, increasin faction loyalt...",notebook,action,8960412060009,8960412060009
5,betterliving,0.3,literacy,researching advances in medication and domesti...,notebook,action,9395913208376,9395913208376
6,individual_education,0.2,wealth,"expand privatized education, increase literacy...",notebook,action,1129902930373,1129902930373


These are the actions available to take.